In [148]:
#import libraries
import numpy as np
import pandas as pd 
import re
import random 
import itertools
import matplotlib.pyplot as plt
from pandas import DataFrame
from nltk.tokenize import word_tokenize
from nltk.stem.isri import ISRIStemmer
from sklearn import tree
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import cross_validation 
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression, SGDClassifier, LogisticRegressionCV
from sklearn.svm import SVC 
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier 
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.naive_bayes import MultinomialNB
from sklearn.utils.multiclass import unique_labels

%matplotlib inline

In [6]:
#file uploads as pandas data frame
xls_file = pd.ExcelFile('//home//rua//Desktop//Ruaa12//redata2.xlsx')
print(xls_file.sheet_names)
data = xls_file.parse('Sheet1')


['Sheet1']


In [7]:
#exploring the data 
print(len(data))
data

4712


,text,Label
0,"قلت لى اقفل التلفون وشغلو تانى,لا لا ظرىف\nسود...",0.0
1,زىن.السودان ممكن نفهم بس القطعة البتحصل دى شنو,-1.0
2,فكرة دعاىة اضغط نجمة وشىل النغمة دى ما جمىلة...,-1.0
3,كىف احول رصىد فى زىن ىا جماعة؟ زىن.السودان,0.0
4,اعلانات زىن، شركة اتصالات ولا ماركة ملابس. ز...,-1.0
5,سودانى اشتراك شهر نت..زىن اشتراك اسبوع\nارىبا ...,-1.0
6,قرر ىبارك لرمضان من اسى عشان النت زادوا ما اظن...,-1.0
7,ى رب نت سودانى ىتنشط,0.0
8,\nبالمقارنه بزىن اسعارهم كوىسه. .,1.0
9,\nمعلىش \nخش فى نت سودانى خىاراتهم زابطه.,1.0


In [15]:
#Stop word removing function, stop words is the useless words in english such as a, an .
def stopwordRemove(text):
    ar_stopword_list = open("//home//rua//Desktop//Ruaa12//stopwords_dictionary.txt", "r")
    stop_words = ar_stopword_list.read().split('\n')
    needed_words = []
    words = word_tokenize(text)
    for w in words:
        if w not in (stop_words):
            needed_words.append(w)
    filtered_sentence = " ".join(needed_words)
    return filtered_sentence

In [21]:
#upload the stop word file
ar_sw_file = open("//home//rua//Desktop//Ruaa12//stopwords_dictionary.txt", "r", encoding = 'utf-8')
ar_list = ar_sw_file.read()
print(stopwordRemove("دوما هي الافضل"))

الافضل


In [22]:
#function to reduce the words to the root word
def steaming(text):
    st = ISRIStemmer()
    stemmed_words = []
    words = word_tokenize(text)
    for w in words:
        stemmed_words.append(st.stem(w))
    stemmed_sentence = ' '.join(stemmed_words)
    return stemmed_sentence
print(steaming("دوما هي الافضل"))

دوم هي فضل


In [23]:
#function to remove the stop-words, steaming 
def prepareDataSet(data):
    sent = []
    for index, r in data.iterrows():
        text = stopwordRemove(r['text'])
        text = steaming(r['text'])
        if r['Label'] == -1:
            sent.append([text, 'neg'])
        elif r['Label'] == 1:
            sent.append([text, 'pos'])
        elif r['Label'] == 0:
            sent.append([text, 'obj'])
    df_sent = DataFrame(sent, columns = ['text','Label'])
    return df_sent
prepareDataSet_data = prepareDataSet(data)

In [24]:
#function to return term frequency–inverse document frequency witch is 
#a numerical statistic that is intended to reflect how important a word is to a document in a collection or corpus.
def FeatureExtraction(data):
    vectorizer = TfidfVectorizer(min_df = 10, lowercase = False,max_df = 0.75, ngram_range = (1, 2))
    tfidf_data = vectorizer.fit_transform(data)
    return tfidf_data

In [25]:
def learing(clf, X, Y):
    X_train, X_test, Y_train, Y_test = \
    cross_validation.train_test_split(X,Y, test_size = 0.4, random_state = 43)
     
    classifer = clf()
    classifer.fit(X_train, Y_train)
    predict = cross_validation.cross_val_predict(classifer, X_test, Y_test, cv=10)
    score = cross_validation.cross_val_score(classifer, X_test, Y_test, cv=10)
    print(score)
    

In [26]:
def main(clf):
    xls_file = pd.ExcelFile('//home//rua//Desktop//Ruaa12//redata2.xlsx')
    data = xls_file.parse('Sheet1')
    prepareDataSet_data = prepareDataSet(data)
    data, target = prepareDataSet_data['text'], prepareDataSet_data['Label']
    tfidf_data = FeatureExtraction(data)
    learing(clf, tfidf_data, target)
    
    

In [27]:
clfs = [MultinomialNB, SVC, LogisticRegression]

In [28]:
for clf in clfs:
    main(clf)

[0.70526316 0.71052632 0.68947368 0.7037037  0.72340426 0.7287234
 0.71808511 0.73404255 0.71657754 0.73796791]
[0.7        0.7        0.7        0.7037037  0.70744681 0.70744681
 0.70744681 0.70744681 0.70588235 0.70588235]
[0.73157895 0.71578947 0.71052632 0.71957672 0.73404255 0.7287234
 0.7287234  0.7393617  0.71657754 0.73262032]


In [30]:
xls_file = pd.ExcelFile('//home//rua//Desktop//Ruaa12//redata2.xlsx')
data = xls_file.parse('Sheet1')
prepareDataSet_data = prepareDataSet(data)
data, target = prepareDataSet_data['text'], prepareDataSet_data['Label']
tfidf_data = FeatureExtraction(data)
learing(clf, tfidf_data, target)    

[0.73157895 0.71578947 0.71052632 0.71957672 0.73404255 0.7287234
 0.7287234  0.7393617  0.71657754 0.73262032]


# KNN


In [54]:
#Grid search the hyper-parameter for KNN
knn_clf = KNeighborsClassifier()
param_knn = {
    'n_neighbors': [1, 2]
}
knn_grid = GridSearchCV(knn_clf, param_knn, cv = 3, scoring = 'f1_micro', n_jobs=2)


In [123]:
%time
score = knn_grid.fit(tfidf_data, target)
score.best_params_

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.77 µs


{'n_neighbors': 2}

In [124]:
score.best_score_
knn_clf = score.best_estimator_
pred_y = knn_clf.predict(tfidf_data)
accuracy_score(target, pred_y)

0.9229462959032053

In [62]:
confusion_matrix(target, pred_y)

array([[3346,    3,    8],
       [ 154,  480,    4],
       [ 122,   71,  523]])

In [144]:

def plot_confusion_matrix(y_true, y_pred, classes,
                          normalize=False,
                          title=None,
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if not title:
        if normalize:
            title = 'Normalized confusion matrix'
        else:
            title = 'Confusion matrix, without normalization'

    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    # Only use the labels that appear in the data
    classes = classes[unique_labels(target, pred_y)]
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    fig.tight_layout()
    return ax

In [146]:

np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plot_confusion_matrix(target, pred_y, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plot_confusion_matrix(target, pred_y, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

NameError: name 'class_names' is not defined

# Naive Bayes

In [68]:
mltiNB_clf = MultinomialNB()
param_MltiNB = {
    'alpha': [0.0, 0.2, 0.5, 0.6, 0.8, 1.0]
    
}
MultiNB_grid = GridSearchCV(mltiNB_clf, param_MltiNB, cv = 3, scoring = 'f1_micro', n_jobs=3)

In [69]:
%time
MultiNB_score = MultiNB_grid.fit(tfidf_data, target)

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 7.39 µs


/home/rua/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/rua/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/rua/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)
/home/rua/anaconda3/lib/python3.7/site-packages/sklearn/naive_bayes.py:472: UserWarning: alpha too small will result in numeric errors, setting alpha = 1.0e-10
  'setting alpha = %.1e' % _ALPHA_MIN)


In [70]:
MultiNB_score.best_score_


0.7431543196773509

In [71]:
MultiNB_score.best_params_

{'alpha': 0.0}

In [73]:
bestMltiNB_clf = MultiNB_score.best_estimator_
predNB_y = bestMltiNB_clf.predict(tfidf_data)
accuracy_score(target,predNB_y)


0.7970706856293781

In [75]:
confusion_matrix(target, predNB_y)

array([[3299,   17,   41],
       [ 467,  143,   28],
       [ 389,   14,  313]])

# SVM

In [79]:
svm_clf = SVC()

In [84]:
param_svm = {
    'kernel':('linear', 'rbf'),
    'C':[5, 10, 15]
}
svm_grid = GridSearchCV(svm_clf, param_svm, cv=3, scoring = 'f1_micro', n_jobs=4)

In [85]:
%%time
svm_score = svm_grid.fit(tfidf_data, target)

CPU times: user 1.55 s, sys: 47.2 ms, total: 1.59 s
Wall time: 7.55 s


In [86]:
svm_score.best_score_

0.7323285926554871

In [87]:
svm_score.best_params_

{'C': 5, 'kernel': 'linear'}

In [88]:
bestsvm_clf = svm_score.best_estimator_
predSVM_y = bestsvm_clf.predict(tfidf_data)
accuracy_score(target, predSVM_y)

0.8868605391636595

In [90]:
confusion_matrix(target, predSVM_y)

array([[3289,   30,   38],
       [ 253,  338,   47],
       [ 150,   15,  551]])

# Logistic Regression

In [111]:
log_clf = LogisticRegression()

In [113]:
param_logistic = {
    #'penalty':('12'),
    'C':[0.2, 0.5, 0.6, 0.8, 1.0]
}
log_grid = GridSearchCV(log_clf, param_logistic, cv=3, scoring = 'f1_micro')


In [114]:
%%time
log_score = log_grid.fit(tfidf_data, target)

CPU times: user 431 ms, sys: 0 ns, total: 431 ms
Wall time: 428 ms


In [115]:
log_score.best_score_

0.7414561664190193

In [116]:
log_score.best_params_

{'C': 1.0}

In [120]:
best_clf = log_score.best_estimator_
predLOG_y = best_clf.predict(tfidf_data)

In [121]:
confusion_matrix(target, predLOG_y)

array([[3331,    4,   22],
       [ 521,   83,   34],
       [ 407,   15,  294]])